### All the Required Imports Goes in to this Cell

In [7]:
import polars as pl
from polars import DataFrame
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime as dt

### Reading the input dataset `prices-split-adjusted.csv`

In [22]:
df: DataFrame = pl.read_csv("prices-split-adjusted.csv")
df.head()

date,symbol,open,close,low,high,volume
str,str,f64,f64,f64,f64,f64
"""2016-01-05""","""WLTW""",123.43,125.839996,122.309998,126.25,2.1636e6
"""2016-01-06""","""WLTW""",125.239998,119.980003,119.940002,125.540001,2.3864e6
"""2016-01-07""","""WLTW""",116.379997,114.949997,114.93,119.739998,2.4895e6
"""2016-01-08""","""WLTW""",115.480003,116.620003,113.5,117.440002,2.0063e6
"""2016-01-11""","""WLTW""",117.010002,114.970001,114.089996,117.330002,1.4086e6


### Converting the date from `string` to actual `date` type

In [23]:
df = df.with_columns([
    pl.col("date").str.to_date().alias("date")
]).sort(["symbol","date"])
df.head()

date,symbol,open,close,low,high,volume
date,str,f64,f64,f64,f64,f64
2010-01-04,"""A""",22.453504,22.389128,22.267525,22.62518,3.8155e6
2010-01-05,"""A""",22.324749,22.145923,22.002861,22.331903,4.186e6
2010-01-06,"""A""",22.06724,22.06724,22.002861,22.174536,3.2437e6
2010-01-07,"""A""",22.017168,22.038626,21.816881,22.04578,3.0951e6
2010-01-08,"""A""",21.917024,22.031474,21.74535,22.06724,3.7339e6


### Filling Missing date records with forward fill

In [20]:
df = df.upsample(time_column="date",every="1d",group_by=["symbol"]).select(pl.all().forward_fill()).sort(["symbol","date"])
df.head(300)

date,symbol,open,close,low,high,volume
date,str,f64,f64,f64,f64,f64
2016-01-05,"""WLTW""",123.43,125.839996,122.309998,126.25,2.1636e6
2016-01-06,"""WLTW""",125.239998,119.980003,119.940002,125.540001,2.3864e6
2016-01-07,"""WLTW""",116.379997,114.949997,114.93,119.739998,2.4895e6
2016-01-08,"""WLTW""",115.480003,116.620003,113.5,117.440002,2.0063e6
2016-01-09,"""WLTW""",115.480003,116.620003,113.5,117.440002,2.0063e6
…,…,…,…,…,…,…
2016-10-26,"""WLTW""",126.129997,126.110001,125.360001,126.919998,478300.0
2016-10-27,"""WLTW""",126.650002,125.629997,125.459999,127.160004,417000.0
2016-10-28,"""WLTW""",125.889999,125.650002,124.519997,126.720001,445700.0
